# Smart AI-Powered Nutrition Assistant

In an era where health awareness is growing, individuals increasingly seek personalized nutrition guidance. However, most existing tools provide generic diet plans, lack real-time adaptability, and fail to consider a person's holistic lifestyle, cultural preferences, allergies, and evolving health conditions. Furthermore, dieticians and nutritionists face limitations in scaling personalized consultations due to time and resource constraints.

Generative AI presents a groundbreaking opportunity to revolutionize this space by enabling an intelligent, interactive, and adaptive virtual nutrition assistant. By leveraging natural language processing (NLP), multimodal understanding, and large-scale dietary databases, an AI-powered assistant can generate dynamic meal plans, recommend smart food swaps, and explain nutritional choices—all tailored to the individual.

This project aims to develop “The Smartest AI Nutrition Assistant” using state-of-the-art generative AI models that:

· Understand user inputs via text, voice, or image (e.g., food photos, grocery labels)

· Generate personalized meal plans based on health goals, medical conditions, fitness routines, and preferences

· Offer contextual explanations (e.g., “Why is this food better?”)

· Adapt suggestions dynamically with continuous feedback

By integrating health data, food databases, and LLM-powered reasoning, the solution will bridge the gap between one-size-fits-all diet apps and in-person nutrition counselling—delivering an AI that thinks, learns, and cares like a real nutrition expert.

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

### Agents

profile_analyzer = Agent(
    role="User Profile Analyzer",
    goal="Analyze the user's health and lifestyle data to identify key factors that should influence their nutritional planning.",
    backstory="You are a data-driven analyst specializing in health and nutrition personalization. "
              "Your job is to interpret raw user input — such as age, gender, weight, height, medical conditions, allergies, and preferences — "
              "and extract meaningful insights that will guide the Nutrition Assistant. "
              "You focus on detecting any health risks, contraindications, or notable patterns. "
              "You do not generate meal plans yourself, but you inform the Nutrition Assistant of all relevant considerations. "
              "You are especially sensitive to chronic conditions, extreme values (e.g., underweight or obesity), and conflicting dietary restrictions.",
    allow_delegation=False,
    verbose=True
)

meal_planner = Agent(
    role="Meal Planner",
    goal="Design a personalized weekly meal plan aligned with the user's dietary preferences, goals, and health requirements.",
    backstory="You are an expert in nutrition and culinary planning. "
              "You use data provided by the User Profile Analyzer and Nutrition Assistant to create well-balanced, satisfying, and diverse meals. "
              "You consider dietary preferences (e.g., vegetarian, vegan), religious/cultural restrictions, allergies, and caloric needs. "
              "You ensure each meal is nutritionally complete and sustainable long-term. "
              "You provide practical meal suggestions that can be prepared easily or sourced locally. "
              "You clearly label meals by time (breakfast, lunch, dinner, snacks) and include estimated calories.",
    allow_delegation=False,
    verbose=True
)

grocery_list_generator = Agent(
    role="Grocery List Generator",
    goal="Generate a complete and organized grocery shopping list based on the user's personalized weekly meal plan.",
    backstory="You are a utility-focused assistant that helps users implement their meal plans by creating efficient grocery lists. "
              "You analyze the meals planned by the Meal Planner and compile the necessary ingredients. "
              "You organize the list by food categories (e.g., produce, grains, dairy, proteins, spices). "
              "You minimize waste by consolidating similar items, optimizing for quantity and storage. "
              "You can optionally suggest sustainable or local alternatives, or flag uncommon items that may need substitution.",
    allow_delegation=False,
    verbose=True
)

health_coach = Agent(
    role="Health Coach",
    goal="Motivate the user and provide actionable wellness tips aligned with their nutrition and health goals.",
    backstory="You are a friendly and supportive virtual coach dedicated to helping users stay on track with their health journey. "
              "You use insights from the Nutrition Assistant and Profile Analyzer to provide habit-forming suggestions, hydration reminders, "
              "mental wellness tips, and progress encouragement. "
              "You speak in a warm, empowering tone and celebrate small wins. "
              "You are not a medical professional, but you are backed by behavioral science and health coaching frameworks.",
    allow_delegation=False,
    verbose=True
)

progress_tracker = Agent(
    role="Progress Tracker",
    goal="Track the user’s nutrition and health progress over time and provide periodic insights or alerts.",
    backstory="You are a data-focused assistant who monitors the user's self-reported progress. "
              "You track metrics such as weight, energy levels, food adherence, mood, hydration, and physical activity. "
              "You compare the current state to the user's original profile and goals. "
              "You generate periodic summaries (weekly/monthly) and notify the Nutrition Assistant or Health Coach of any notable changes. "
              "You highlight both improvements and areas of concern. "
              "You ensure privacy and data accuracy while supporting the long-term success of the user.",
    allow_delegation=False,
    verbose=True
)

recipe_recommender = Agent(
    role="Recipe Recommender",
    goal="Suggest healthy, personalized recipes based on available ingredients, preferences, and goals.",
    backstory="You are a culinary-savvy assistant with access to a global recipe database. "
              "You match ingredients from the user's grocery list or kitchen inventory to suggest easy-to-make, healthy recipes. "
              "You consider dietary preferences, portion size, cooking skills, and cultural food familiarity. "
              "You highlight prep time, cooking time, and nutritional breakdown. "
              "You can also recommend smart substitutions and budget-friendly hacks.",
    allow_delegation=False,
    verbose=True
)


### Tasks

In [18]:
profile_analysis_task = Task(
    description=(
        "1. Analyze the user's profile data including age, gender, weight, height, "
        "medical conditions, allergies, dietary preferences, and activity level.\n"
        "2. Identify key health risks or factors that will impact nutrition planning.\n"
        "3. Summarize any important contraindications or special considerations.\n"
        "4. Provide clear insights to guide the nutrition assistant agent.\n"
    ),
    expected_output=
        "A detailed analysis report highlighting important health and lifestyle "
        "factors relevant to nutrition personalization.",
    agent=profile_analyzer,
)

meal_planning_task = Task(
    description=(
        "1. Use the user's analyzed profile data to create a balanced weekly meal plan.\n"
        "2. Ensure meals align with dietary preferences, allergies, and nutritional goals.\n"
        "3. Include breakfast, lunch, dinner, and snack suggestions with calorie estimates.\n"
        "4. Incorporate variety and culturally appropriate foods.\n"
        "5. Provide simple preparation tips or notes if relevant.\n"
    ),
    expected_output=
        "A complete weekly meal plan formatted with meals by day and time, "
        "including nutritional highlights and calorie counts.",
    agent=meal_planner,
)

grocery_list_task = Task(
    description=(
        "1. Analyze the weekly meal plan to extract all required ingredients.\n"
        "2. Organize ingredients into logical grocery categories (e.g., produce, dairy).\n"
        "3. Optimize quantities to minimize waste and ensure practicality.\n"
        "4. Suggest sustainable or local alternatives where applicable.\n"
    ),
    expected_output=
        "An organized grocery shopping list sorted by category, "
        "ready for use by the user.",
    agent=grocery_list_generator,
)

health_coach_task = Task(
    description=(
        "1. Review the user's nutrition and health goals and progress data.\n"
        "2. Provide motivational advice and habit-forming tips tailored to the user.\n"
        "3. Suggest small, actionable wellness steps related to nutrition and lifestyle.\n"
        "4. Use an encouraging and positive tone throughout.\n"
    ),
    expected_output=
        "A personalized coaching message designed to support and motivate the user "
        "in achieving their health goals.",
    agent=health_coach,
)

progress_tracker_task = Task(
    description=(
        "1. Monitor user-submitted data over time, such as weight, energy, and adherence.\n"
        "2. Compare progress against initial goals and profiles.\n"
        "3. Highlight notable improvements and flag areas needing attention.\n"
        "4. Generate periodic summary reports for the user and health coach.\n"
    ),
    expected_output=
        "Clear, concise progress reports that provide insights into the user's "
        "nutrition and wellness journey.",
    agent=progress_tracker,
)

recipe_recommender_task = Task(
    description=(
        "1. Use the user's available ingredients and dietary preferences to find matching recipes.\n"
        "2. Prioritize recipes that are healthy, quick to prepare, and culturally appropriate.\n"
        "3. Provide nutritional information, preparation time, and serving sizes.\n"
        "4. Suggest ingredient substitutions for allergies or preferences.\n"
    ),
    expected_output=
        "A curated list of personalized recipes with detailed instructions "
        "and nutritional info.",
    agent=recipe_recommender,
)


## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [22]:
crew = Crew(
    agents=[profile_analyzer, meal_planner, grocery_list_generator, health_coach, progress_tracker, recipe_recommender],
    tasks=[profile_analysis_task, meal_planning_task, grocery_list_task, health_coach_task, recipe_recommender_task],
    verbose=2
)

2025-05-18 07:13:44,171 - 140511159036800 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [23]:
result = crew.kickoff(inputs={"user_profile": "28-year-old vegetarian female, 160cm, 65kg, goal: weight loss, high activity, prefers Indian cuisine"})

 [DEBUG]: == Working Agent: User Profile Analyzer
 [INFO]: == Starting Task: 1. Analyze the user's profile data including age, gender, weight, height, medical conditions, allergies, dietary preferences, and activity level.
2. Identify key health risks or factors that will impact nutrition planning.
3. Summarize any important contraindications or special considerations.
4. Provide clear insights to guide the nutrition assistant agent.



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

User Profile Data Analysis Report:

Age: 45
Gender: Female
Weight: 170 lbs
Height: 5'6"
Medical Conditions: Hypothyroidism, Type 2 Diabetes
Allergies: None reported
Dietary Preferences: Vegetarian
Activity Level: Light exercise 3 times a week

Key Health Risks or Factors:
1. Hypothyroidism and Type 2 Diabetes are significant factors that will impact nutrition planning. Both conditions require careful management of blood sugar levels and may necessitate specific diet

I now can give a great answer

Final Answer:
**Organized Grocery Shopping List:**

**Produce:**
- Mixed berries
- Fresh berries
- Chia seeds
- Avocado
- Grapefruit
- Spinach
- Bell peppers
- Zucchini
- Mango
- Sweet potato
- Black beans
- Salsa ingredients (tomato, onion, cilantro)
- Guacamole ingredients (avocado, lime, tomato)
- Apple
- Cherry tomatoes
- Green beans
- Pineapple
- Fresh herbs (for garnish)
- Variety of salad greens
- Falafel ingredients (chickpeas, herbs)
- Roasting vegetables (cauliflower, broccoli)

**Grains & Legumes:**
- Quinoa
- Brown rice
- Whole grain bread
- Whole wheat pasta
- Oatmeal
- Rice cakes
- Whole grain pancakes mix

**Dairy & Alternatives:**
- Greek yogurt
- Almond milk
- Vegan yogurt
- Cottage cheese

**Proteins:**
- Lentils
- Chickpeas
- Tofu
- Vegan meatballs
- Almonds
- Dried apricots
- Nuts (walnuts, trail mix)
- Plant-based protein powder

**Spices & Condiments:**
- Hummus
- Nut butter (peanut, almond)
- Tahini
- Maple syrup
- Various herbs and

- Display the results of your execution as markdown in the notebook.

In [24]:
from IPython.display import Markdown
Markdown(result)

Recipe 1: Lentil and Chickpea Curry
Ingredients:
- 1 cup lentils
- 1 can chickpeas
- 1 onion, diced
- 2 cloves garlic, minced
- 1 can diced tomatoes
- 1 can coconut milk
- 1 tbsp cumin
- 1 tbsp coriander
- 1 tbsp turmeric
- Salt and pepper to taste

Instructions:
1. In a large pot, sauté the onion and garlic until fragrant.
2. Add the lentils, chickpeas, diced tomatoes, coconut milk, and spices. Stir well.
3. Simmer for 20-25 minutes until lentils are tender.
4. Season with salt and pepper to taste.
5. Serve over rice or quinoa.

Nutritional Info:
- Calories: 350 per serving
- Protein: 15g
- Fiber: 10g

Recipe 2: Tofu Stir-Fry
Ingredients:
- 1 block tofu, cubed
- Mixed vegetables (bell peppers, broccoli, snap peas)
- 2 tbsp soy sauce
- 1 tbsp sesame oil
- 1 tsp ginger, minced
- 1 tsp garlic, minced

Instructions:
1. In a large pan, sauté the tofu until golden brown.
2. Add in the mixed vegetables, ginger, and garlic. Stir-fry for 5-7 minutes.
3. Pour in soy sauce and sesame oil. Stir well to combine.
4. Cook for an additional 2-3 minutes.
5. Serve over brown rice or noodles.

Nutritional Info:
- Calories: 250 per serving
- Protein: 12g
- Fiber: 5g

Substitution Suggestions:
- For allergies to soy, use coconut aminos as a soy sauce alternative.
- For preferences, swap tofu with tempeh or seitan in the stir-fry recipe.

Enjoy these plant-based protein-packed recipes tailored to your dietary preferences and health goals!